# pandas高级操作
- 1.替换操作
    - 可以同步作用于Series和DataFrame中
    - 单值替换
        - 普通替换：替换所有符合要求的元素：to_replace = 15,value='3'
        - 按列指定单值替换：to_replace={列标签：替换值} value='value'
    - 多值替换
        - 列表替换：to_replace=[] value=[]
        - 字典替换（推荐） to_replace = {to_replace:value,to_replace:value}
- 2.映射操作 map
    - 概念：创建一个映射关系列表，把values元素和一个特定的标签或者字符串绑定（给一个元素值提供不同的表现形式）
    - 创建一个df,两列分别是姓名和薪资，然后给其他名字起对应的英文名
        - map是series方法，只能被Series调用
            - 1.映射作用
            - 2.运算工具
- 3.排序实现的随机抽样
    - take()
    - np.random.permutation()
- 4.数据分类处理
    - 核心：
        - groupby()函数
        - groups属性查看分组情况
- 5.高级数据聚合
    - 1.使用groupby()分组后，也可以使用transform和apply提供自定义函数实现更多运算
    - 2.df.groupby(by='item')['price'].sum()<==>df.groupby(by='item')['price'].apply(sum)
    - 3.transform和apply都会进行运算，在transform或者apply中传入函数即可
    - 4.transform和apply也可以传入一个lambda表达式





In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

### 替换操作

In [2]:
df = DataFrame(data=np.random.randint(0,100,size=(5,6)))
df

,0,1,2,3,4,5
0,81,33,38,27,56,63
1,8,9,24,42,99,64
2,61,38,10,18,72,99
3,23,43,56,49,39,21
4,46,89,16,6,89,43


In [3]:
df.replace(to_replace=81,value='two')

,0,1,2,3,4,5
0,two,33,38,27,56,63
1,8,9,24,42,99,64
2,61,38,10,18,72,99
3,23,43,56,49,39,21
4,46,89,16,6,89,43


In [4]:
df.replace(to_replace={8:'one'})

,0,1,2,3,4,5
0,81,33,38,27,56,63
1,one,9,24,42,99,64
2,61,38,10,18,72,99
3,23,43,56,49,39,21
4,46,89,16,6,89,43


### 映射操作
map是series方法，只能被Series调用
- 1.映射作用
- 2.运算工具

In [5]:
dic = {
    'name':['张三','李四','王二'],
    'salary':[15000,20000,25000]
}
df = DataFrame(data=dic)
df

,name,salary
0,张三,15000
1,李四,20000
2,王二,25000


In [7]:
# 映射关系表
dic_mapping = {
    '张三':'Tom',
    '李四':'Jack',
    '王二':'Mark'
}
df['e_name'] = df['name'].map(dic_mapping)
df

,name,salary,e_name
0,张三,15000,Tom
1,李四,20000,Jack
2,王二,25000,Mark


In [8]:
def after_sal(s): #计算税后薪资
    return s- (s-3000)*0.5
df['after_sal']=df['salary'].map(after_sal) #可以将df['salary']这个Series中每个元素作为参数传递
df

,name,salary,e_name,after_sal
0,张三,15000,Tom,9000.0
1,李四,20000,Jack,11500.0
2,王二,25000,Mark,14000.0


### 排序实现的随机抽样
- take()
- np.random.permutation()

In [10]:
df = DataFrame(data=np.random.randint(0,100,size=(100,3)),columns=['A','B','C'])
df

,A,B,C
0,17,7,87
1,40,13,40
2,88,70,95
3,90,43,43
4,30,2,18
...,...,...,...
95,45,88,34
96,8,91,72
97,90,95,73
98,49,61,96


In [13]:
# 生成乱序的随机序列
np.random.permutation(3)
# 将源数据打乱
df.take([2,0,1],axis=1) #列
randf = df.take(np.random.permutation(3),axis=1).take(np.random.permutation(99),axis=0) #行列全部打乱
randf[0:50]

,A,B,C
73,75,30,81
8,43,0,17
90,15,8,14
74,28,76,21
46,85,39,26
2,88,70,95
3,90,43,43
83,0,70,70
61,60,19,54
16,89,10,40


## 数据分类处理
- 核心：
    - groupby()函数
    - groups属性查看分组情况

In [32]:
df = DataFrame({
    'item':['Apple','Banana','Orange','Banana','Orange','Apple'],
    'price':[4,3,3,2.5,4,2],
    'color':['red','yellow','yellow','green','green','green'],
    'weight':[12,20,50,30,20,44]
})
df

,item,price,color,weight
0,Apple,4.0,red,12
1,Banana,3.0,yellow,20
2,Orange,3.0,yellow,50
3,Banana,2.5,green,30
4,Orange,4.0,green,20
5,Apple,2.0,green,44


In [21]:
# 对水果种类分组
df.groupby(by='item').groups

{'Apple': [0, 5], 'Banana': [1, 3], 'Orange': [2, 4]}

- 分组聚合

In [33]:
# 计算出每种水果的平均价格
df.groupby(by='item')['price'].mean()

item
Apple     3.00
Banana    2.75
Orange    3.50
Name: price, dtype: float64

In [25]:
# 计算出每种颜色对应水果的平均重量
df.groupby(by='color')['weight'].mean()

color
green     31.333333
red       12.000000
yellow    35.000000
Name: weight, dtype: float64

In [35]:
# 将计算出的平均重量汇总的到源数据中
dic_cw = df.groupby(by='color')['weight'].mean().to_dict()
df['mean_w'] = df['color'].map(dic_cw)
df

,item,price,color,weight,mean_w
0,Apple,4.0,red,12,12.000000
1,Banana,3.0,yellow,20,35.000000
2,Orange,3.0,yellow,50,35.000000
3,Banana,2.5,green,30,31.333333
4,Orange,4.0,green,20,31.333333
5,Apple,2.0,green,44,31.333333


In [39]:
def my_mean(s):
    m_sum=0
    # print(s)
    for i in s:
        m_sum +=i
    return m_sum/ len(s)

In [40]:
df.groupby(by='item')['price'].transform(my_mean)

0    3.00
1    2.75
2    3.50
3    2.75
4    3.50
5    3.00
Name: price, dtype: float64

In [41]:
df.groupby(by='item')['price'].apply(my_mean)

item
Apple     3.00
Banana    2.75
Orange    3.50
Name: price, dtype: float64